# Imputación básica

## Preparar datos

In [190]:
import pandas as pd
import numpy as n
import matplotlib.pyplot as plt
import random
import os
import warnings
import math

## Leer datos originales

Hay que leer el dataset original para comparar los datos imputados con los reales

Temperatura

In [191]:
warnings.filterwarnings('ignore', category=SyntaxWarning)  # Porque el \d de \data y \daily lo toma como un valor especial
dataframe_temperatura = pd.read_csv("..\data\daily-minimum-temperatures-in-me.csv", header=0, index_col=0, parse_dates=True)
series_temperatura = dataframe_temperatura.squeeze('columns')
series_temperatura = series_temperatura.resample('D').mean()
series_temperatura['1984-12-31'] = (series_temperatura['1984-12-30'] + series_temperatura['1985-01-01'])/2
series_temperatura['1988-12-31'] = (series_temperatura['1988-12-30'] + series_temperatura['1989-01-01'])/2
series_temperatura.head()

Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
Freq: D, Name: Daily minimum temperatures in Melbourne, Australia, 1981-1990, dtype: float64

Cardiovascular

In [192]:
dataframe_cardiovascular = pd.read_csv("..\data\\ads_hour.csv", header=0, index_col=0, parse_dates=True)
series_cardiovascular = dataframe_cardiovascular.squeeze('columns')
series_cardiovascular.head()

C:\Users\alaba\AppData\Local\Temp\ipykernel_17464\1350282516.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataframe_cardiovascular = pd.read_csv("..\data\\ads_hour.csv", header=0, index_col=0, parse_dates=True)


Date
2017-08-03 00:00:00    49136
2017-08-03 01:00:00    46450
2017-08-03 02:00:00    46355
2017-08-03 03:00:00    43748
2017-08-03 04:00:00    42281
Name: ads, dtype: int64

Manchas solares

In [193]:
dataframe_sunspots = pd.read_csv("..\data\monthly-sunspots.csv", header=0, index_col=0, parse_dates=True)
series_sunspots = dataframe_sunspots.squeeze('columns')
series_sunspots = series_sunspots.resample('ME').mean() # Para los meses
series_sunspots.head()

Month
1749-01-31    58.0
1749-02-28    62.6
1749-03-31    70.0
1749-04-30    55.7
1749-05-31    85.0
Freq: ME, Name: Sunspots, dtype: float64

## Funciones auxiliares

### Listar dataset

In [194]:
def listar_datasets(ruta):
    return os.listdir(ruta)

### Calcular RMSE

In [195]:
def RMSE(serie_original, serie_nula, serie_imputada):
    suma=0
    for i in serie_nula[serie_nula.isna()].index:
        suma += (serie_original[i] - serie_imputada[i])**2
    suma = math.sqrt(suma/len(serie_original))
    return suma

### Calcular MAE

In [196]:
def MAE(serie_original, serie_nula, serie_imputada):
    suma=0
    for i in serie_nula[serie_nula.isna()].index:
        suma += abs(serie_original[i] - serie_imputada[i])
    suma = suma/len(serie_original)
    return suma

### Calcular MAPE

In [197]:
def MAPE(serie_original, serie_nula, serie_imputada):
    suma=0
    for i in serie_nula[serie_nula.isna()].index:
        if(serie_original[i] != 0):
            suma += float(abs(serie_original[i] - serie_imputada[i]))/serie_original[i]
        #elif(serie_imputada[i] != 0):
            #suma += float(abs(serie_original[i] - serie_imputada[i]))/serie_imputada[i]
    suma = suma/len(serie_original)
    return 100*suma

### Calcular BIAS

In [198]:
def BIAS(serie_original, serie_nula, serie_imputada):
    suma=0
    for i in serie_nula[serie_nula.isna()].index:
        suma += serie_original[i] - serie_imputada[i]
    suma = suma/len(serie_original)
    return suma

### Imputar serie

#### Media

In [199]:
def imputar_media(serie, ruta, lista):
    dataframe_media = pd.DataFrame({
        'Dataset': [],
        'RMSE': [],
        'MAE': [],
        'MAPE': [],
        'BIAS': []
    })
    for i in lista:
        # Lo leemos
        df = pd.read_csv(ruta+i, header=0, index_col=0, parse_dates=True)
        serie_nulo = df.squeeze('columns')
        
        # Imputamos con la media:
        serie_media = serie_nulo.fillna(serie_nulo.mean())

        # Calculamos la raiz del error cuadrático medio con las series imputadas con la media
        rmse_media = RMSE(serie, serie_nulo, serie_media)
        # Calculamos el error medio absoluto con las series imputadas con la media
        mae_media = MAE(serie, serie_nulo, serie_media)
        # Calculamos el error medio absoluto porcentual con las series imputadas con la media
        mape_media = MAPE(serie, serie_nulo, serie_media)
        # Calculamos el bias con las series imputadas con la media
        bias_media = BIAS(serie, serie_nulo, serie_media)

        nueva_fila = pd.DataFrame({ 'Dataset': [i], 'RMSE': [rmse_media], 'MAE': [mae_media], 'MAPE': [mape_media], 'BIAS': [bias_media]})
        dataframe_media = pd.concat([dataframe_media, nueva_fila], ignore_index=True)
    return dataframe_media

#### Mediana

In [200]:
def imputar_mediana(serie, ruta, lista):
    dataframe_mediana = pd.DataFrame({
        'Dataset': [],
        'RMSE': [],
        'MAE': []
    })
    for i in lista:
        # Lo leemos
        df = pd.read_csv(ruta+i, header=0, index_col=0, parse_dates=True)
        serie_nulo = df.squeeze('columns')
        
        # Imputamos con la mediana
        serie_mediana = serie_nulo.fillna(serie_nulo.median())

        # Calculamos la raiz del error cuadrático medio con las series imputadas con la mediana
        rmse_mediana = RMSE(serie, serie_nulo, serie_mediana)
        # Calculamos el error medio absoluto con las series imputadas con la mediana
        mae_mediana = MAE(serie, serie_nulo, serie_mediana)
        # Calculamos el error medio absoluto porcentual con las series imputadas con la mediana
        mape_mediana = MAPE(serie, serie_nulo, serie_mediana)
        # Calculamos el bias con las series imputadas con la mediana
        bias_mediana = BIAS(serie, serie_nulo, serie_mediana)

        nueva_fila = pd.DataFrame({ 'Dataset': [i], 'RMSE': [rmse_mediana], 'MAE': [mae_mediana], 'MAPE': [mape_mediana], 'BIAS': [bias_mediana]})
        dataframe_mediana = pd.concat([dataframe_mediana, nueva_fila], ignore_index=True)
    return dataframe_mediana

## Imputamos media

### Dataset de temperaturas

In [201]:
lista_datasets_temperatura = listar_datasets('..\miss_valued_data_temperatura')
dataframe_temperatura_media = imputar_media(series_temperatura, "..\miss_valued_data_temperatura\\", lista_datasets_temperatura)

### Dataset cardiovascular

In [202]:
lista_datasets_cardiovascular = listar_datasets('..\miss_valued_data_cardiovascular')
dataframe_cardiovascular_media = imputar_media(series_cardiovascular, "..\miss_valued_data_cardiovascular\\", lista_datasets_cardiovascular)

### Dataset manchas solares

In [203]:
lista_datasets_sunspots = listar_datasets('..\miss_valued_data_manchas_solares')
dataframe_manchas_solares_media = imputar_media(series_sunspots, "..\miss_valued_data_manchas_solares\\", lista_datasets_sunspots)

### Juntarlos

In [204]:
dataframe_general_media = pd.concat([dataframe_temperatura_media, dataframe_cardiovascular_media, dataframe_manchas_solares_media], ignore_index=True)

In [205]:
dataframe_general_media

,Dataset,RMSE,MAE,MAPE,BIAS
0,min_temperature_bateria_10.csv,1.474758,0.377155,5.727752,-0.062002
1,min_temperature_bateria_15.csv,1.671954,0.519310,7.166528,-0.049206
2,min_temperature_bateria_20.csv,1.957098,0.716304,10.788942,-0.197963
3,min_temperature_error_medida_10.csv,1.306482,0.338414,4.254983,-0.000623
4,min_temperature_error_medida_15.csv,1.601408,0.505805,7.932482,-0.013763
5,min_temperature_error_medida_20.csv,1.790315,0.646521,9.966390,0.002636
6,min_temperature_interferencias_10.csv,1.350493,0.346774,4.124677,0.018786
7,min_temperature_interferencias_15.csv,1.637666,0.513872,6.318518,-0.021911
8,min_temperature_interferencias_20.csv,1.920747,0.696068,10.335958,0.006435
9,min_temperature_mix_10.csv,1.334668,0.337125,4.099961,-0.006962


## Imputamos mediana

### Dataset temperaturas

In [206]:
lista_datasets_temperatura = listar_datasets('..\miss_valued_data_temperatura')
dataframe_temperatura_mediana = imputar_mediana(series_temperatura, "..\miss_valued_data_temperatura\\", lista_datasets_temperatura)

### Dataset cardiovascular

In [207]:
lista_datasets_cardiovascular = listar_datasets('..\miss_valued_data_cardiovascular')
dataframe_cardiovascular_mediana = imputar_mediana(series_cardiovascular, "..\miss_valued_data_cardiovascular\\", lista_datasets_cardiovascular)

### Dataset manchas solares

In [208]:
lista_datasets_sunspots = listar_datasets('..\miss_valued_data_manchas_solares')
dataframe_manchas_solares_mediana = imputar_mediana(series_sunspots, "..\miss_valued_data_manchas_solares\\", lista_datasets_sunspots)

### Juntarlos

In [209]:
dataframe_general_mediana = pd.concat([dataframe_temperatura_mediana, dataframe_cardiovascular_mediana, dataframe_manchas_solares_mediana], ignore_index=True)

In [210]:
dataframe_general_mediana

,Dataset,RMSE,MAE,MAPE,BIAS
0,min_temperature_bateria_10.csv,1.469475,0.374877,5.635695,-0.047850
1,min_temperature_bateria_15.csv,1.668904,0.517347,7.067163,-0.029915
2,min_temperature_bateria_20.csv,1.949547,0.712610,10.683116,-0.182462
3,min_temperature_error_medida_10.csv,1.306689,0.337733,4.218367,0.007393
4,min_temperature_error_medida_15.csv,1.601013,0.504655,7.856051,0.000219
5,min_temperature_error_medida_20.csv,1.792323,0.645728,9.801534,0.038007
6,min_temperature_interferencias_10.csv,1.353682,0.347015,4.067815,0.034858
7,min_temperature_interferencias_15.csv,1.636779,0.513116,6.255012,-0.006709
8,min_temperature_interferencias_20.csv,1.922886,0.695263,10.169463,0.041046
9,min_temperature_mix_10.csv,1.335002,0.336323,4.024887,0.011733


## Imputamos interpolación